# LASSO Regression with coordinate descent

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located. The details about the datset is given at the initial stage only.

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

# Import useful functions from previous notebook

The below 3 functions are function which were defined while implementing gradient descent algorithm in multiple regression model.

In [1]:
import numpy as np 

In [5]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [6]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)

    return(predictions)

In [10]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix/norms
    return (normalized_features, norms)

# Implementing Coordinate Descent with normalized features

The LASSO cost function is given by the following equation. It needs to be minimized.
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```

In coordinate descentt algorithm, one feature is optimized at a time.Hence optimizing one `w[i]` at a time, circling through the weights multiple times. The algorithm looks like below.
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

## Effect of L1 penalty

Let's consider a simple model with 2 features:

In [12]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [13]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [14]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [15]:
prediction = predict_output(simple_feature_matrix, weights)

In [16]:
ro = [0 for i in range((simple_feature_matrix.shape)[1])]
for j in range((simple_feature_matrix.shape)[1]):   
    ro[j] = (simple_feature_matrix[:,j] * (output - prediction + (weights[j] * simple_feature_matrix[:,j]))).sum()
print ro

[79400300.034929156, 87939470.772991076, 80966698.675965652]


In [32]:
print 2* ro[1]

175878941.546


In [33]:
print 2* ro[2]

161933397.352


In [40]:
def in_l1range(value, penalty):
    return ( (value >= -penalty/2.) and (value <= penalty/2.) )

In [55]:
for l1_penalty in [1.4e8, 1.64e8, 1.73e8, 1.9e8, 2.3e8]:
    print in_l1range(ro[1], l1_penalty), in_l1range(ro[2], l1_penalty)

False False
False True
False True
True True
True True


`ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

In [156]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i] * (output - prediction + (weights[i] * feature_matrix[:,i]))).sum()

    if i == 0:
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = (ro_i + l1_penalty/2.)
    elif ro_i > l1_penalty/2.:
        new_weight_i = (ro_i - l1_penalty/2.)
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [40]:
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

In [116]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    D = feature_matrix.shape[1]
    weights = np.array(initial_weights)
    change = np.array(initial_weights) * 0.0
    converged = False

    while not converged:
        for idx in range(D):
            new_weight = lasso_coordinate_descent_step(idx, feature_matrix,
                                                       output, weights,
                                                       l1_penalty)
            change[idx] = np.abs(new_weight - weights[idx])
            weights[idx] = new_weight
        max_change = max(change)
        if max_change < tolerance:
            converged = True
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [117]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [118]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [119]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [120]:
print weights

[ 21624998.36636292  63157246.78545421         0.        ]


In [122]:
prediction =  predict_output(normalized_simple_feature_matrix, weights)
RSS = np.dot(output-prediction, output-prediction)
print 'RSS for normalized dataset = ', RSS

RSS for normalized dataset =  1.63049248148e+15


# Evaluating LASSO fit with more features

Splitting the whole data into 2 parts : training data which is used to train the model and test data which is used to evaluate the performance of the trained model. I am taking training data to be 80% of the total data and hence test data would be the remaining 20% of the total data.

In [125]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [126]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

Firstly, i will create a normalized feature matrix from the TRAINING data with these features

In [127]:
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [128]:
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 1e7
tolerance = 1.0

In [129]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)

In [130]:
print weights1e7

[ 24429600.60933314         0.                 0.          48389174.35227978
         0.                 0.           3317511.16271982   7329961.9848964
         0.                 0.                 0.                 0.
         0.                 0.        ]


In [131]:
feature_list = ['constant'] + all_features
print feature_list

['constant', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


In [132]:
feature_weights1e7 = dict(zip(feature_list, weights1e7))
for k,v in feature_weights1e7.iteritems():
    if v != 0.0:
        print k, v

sqft_living 48389174.3523
waterfront 3317511.16272
constant 24429600.6093
view 7329961.9849


sqft_living, wavefront, constant and view are four nonzero wieghted features.

**If  `l1_penalty=1e8`,  `tolerance=1` : **

In [133]:
l1_penalty=1e8
tolerance = 1.0
weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)

In [134]:
print weights1e8

[ 71114625.75280938         0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.        ]


In [135]:
feature_weights1e8 = dict(zip(feature_list, weights1e8))
for k,v in feature_weights1e8.iteritems():
    if v != 0.0:
        print k, v

constant 71114625.7528


**Only constant feature had non zero weight in this case.**

** If  `l1_penalty=1e4`, `tolerance=5e5`  : **

In [136]:
l1_penalty=1e4
tolerance=5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                               initial_weights, l1_penalty, tolerance)

In [137]:
print weights1e4

[ 77779073.91265225 -22884012.25023361  15348487.08089996
  92166869.69883074  -2139328.0824278   -8818455.54409492
   6494209.73310655   7065162.05053198   4119079.21006765
  18436483.52618776 -14566678.54514342  -5528348.75179426
 -83591746.20730537   2784276.46012858]


In [138]:
feature_weights1e4 = dict(zip(feature_list, weights1e4))
for k,v in feature_weights1e4.iteritems():
    if v != 0.0:
        print k, v

bathrooms 15348487.0809
sqft_above -14566678.5451
grade 18436483.5262
yr_renovated 2784276.46013
bedrooms -22884012.2502
sqft_living 92166869.6988
floors -8818455.54409
condition 4119079.21007
waterfront 6494209.73311
constant 77779073.9127
sqft_basement -5528348.75179
yr_built -83591746.2073
sqft_lot -2139328.08243
view 7065162.05053


**The above all features had non-zero weights.**

## Rescaling learned weights

I normalized our feature matrix, before learning the weights.  To use these weights on a test set, i must now  normalize the test data in the same way.

i am crating normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [93]:
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

In [96]:
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms
normalized_weights1e4 = weights1e4 / norms
print normalized_weights1e7[3]

161.317456248


## Evaluating each of the learned models on the test data

 Let's evaluate all the three models on the test data .

In [95]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [98]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e7)
RSS = np.dot(test_output-prediction, test_output-prediction)
print 'RSS for model with weights1e7 = ', RSS

RSS for model with weights1e7 =  2.75962079909e+14


In [99]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e8)
RSS = np.dot(test_output-prediction, test_output-prediction)
print 'RSS for model with weights1e8 = ', RSS

RSS for model with weights1e8 =  5.37166150034e+14


In [100]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e4)
RSS = np.dot(test_output-prediction, test_output-prediction)
print 'RSS for model with weights1e4 = ', RSS

RSS for model with weights1e4 =  2.2778100476e+14
